### Novas perguntas do CEO para vocês



1. Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre `R$ 0 e R$ 321.950` 
- Nível 1 -> Preço entre `R$ 321.950 e R$ 450.000`
- Nível 2 -> Preço entre `R$450.000 e R$ 645.000`
- Nível 3 -> Acima de `R$ 645.000`

2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ? 
- Size 0 -> Tamanho entre 0 e 1427 sqft 
- Size 1 -> Tamanho entre 1427 e 1910 sqft 
- Size 2 -> Tamanho entre 1910 e 2550 sqft 
- Size 3 -> Tamanho acima de 2550 sqft

3. Adicione as seguinte informações ao conjunto de dados original: 
- Place ID: Identificação da localização 
- OSM Type: Open Street Map type - Country: Nome do País 
- Country Code: Código do País

4. Adicione os seguinte filtros no Mapa: 
- Tamanho mínimo da área da sala de estar. 
- Número mínimo de banheiros. 
- Valor Máximo do Preço. 
- Tamanho máximo da área do porão. 
- Filtro das Condições do Imóvel. 
- Filtro por Ano de Construção.

5. Adicione os seguinte filtros no Dashboard: 
- Filtro por data disponível para compra. 
- Filtro por ano de renovação. 
- Filtro se possui vista para a água ou não.

In [1]:
# Bibliotecas
import numpy as np
import pandas as pd
import time
import ipywidgets as widgets
import plotly.express as px
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib import gridspec
from ipywidgets import interact, interactive, fixed, interact_manual
from geopy.geocoders import Nominatim
from multiprocessing import Pool

In [2]:
# Carregando dados
data = pd.read_csv(r'C:\Users\mlsil\OneDrive\Documentos\repos\python_zero_ao_ds\csv\kc_house_data.csv')

#### 1. Qual a média do preço de compra dos imóveis por “Nível”?
- Nível 0 -> Preço entre `R$ 0 e R$ 321.950`
- Nível 1 -> Preço entre `R$ 321.950 e R$ 450.000`
- Nível 2 -> Preço entre `R$ 450.000 e R$ 645.000`
- Nível 3 -> Acima de `R$ 645.000`

In [3]:
data['level'] = data['price'].apply(lambda x: 'lv00' if (x >= 0) & (x <= 321950) else
                                              'lv01' if (x > 321950) & (x <= 450000) else
                                              'lv02' if (x > 450000) & (x <= 645000) else 'lv03')

In [4]:
#data.head()

#### 2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

In [5]:
data['size'] = data['sqft_living'].apply(lambda x: 's00' if (x >= 0) & (x <= 1427) else
                                                   's01' if (x > 1427) & (x <= 1910) else
                                                   's02' if (x > 1910) & (x <= 2550) else 's03')

In [6]:
#data.head()

#### 3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

In [7]:
# Bibliotecas: 'geocoders' / 'time' / 'Pool'
geolocator = Nominatim(user_agent='geopyExercises')

query = '47.5112, -122.257'
response = geolocator.reverse(query)

In [8]:
# Criando nova coluna para a concatenação de 'lat' e 'long'
data['query'] = data[['lat', 'long']].apply( lambda x: str(x['lat']) + ',' + str(x['long']), axis=1)

In [9]:
# import time
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent='geopyExercises')

# def get_data( x ):
#     index, row = x
#     time.sleep( 1 )
    
#     # chamada API
#     response = geolocator.reverse(row['query'])
#     address = response.raw['address']
    
    
#     place_id = response.raw['place_id'] if 'place_id' in response.raw else 'NA'
#     osm_type = response.raw['osm_type'] if 'osm_type' in response.raw else 'NA'
#     country = address['country'] if 'country' in address else 'NA'
#     country_code = address['country_code'] if 'country_code' in address else 'NA'

#     return place_id, osm_type, country, country_code

In [10]:
# O jupyter não é preparado para rodar arquivos multitrending, por isso , precisa criar um arquivo de 
# forma paralela
import defs
df1 = data[['id', 'query']].sample(10)

p = Pool(3)

start = time.process_time()
df1[['place_id', 'osm_type', 'contry', 'country_code']] = p.map(defs.get_data, df1.iterrows())
end = time.process_time()

print('Time Elapsed: {}', end - start)

Time Elapsed: {} 0.0


In [11]:
df1.head()

,id,query,place_id,osm_type,contry,country_code
16816,8029500380,"47.3066,-122.394",235563807,way,United States,us
16962,4239400840,"47.3161,-122.182",236412275,way,United States,us
2624,3275890310,"47.6496,-122.083",185604295,way,United States,us
12943,1623089025,"47.481,-121.788",111412049,way,United States,us
10019,2473420070,"47.4514,-122.159",233967029,way,United States,us


#### 4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.

In [12]:
# Bibliotecas ipywidgets / import interact, interactive, fixed, interact_manual
#data['is_waterfront'] = data['is_waterfront'].apply( lambda x: 'yes' if x == 1 'no')

# define intercalavite buttons
price_limit = widgets.IntSlider(
    value = int( data['price'].mean() ),
    min = int( data['price'].min() ),
    max = int( data['price'].max() ),
    step = 3,
    description = 'Maximum Price',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
)
# define interactive buttons
living_limit = widgets.IntSlider(
    value = int( data['sqft_living'].mean() ),
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step = 1,
    description = 'Minium Living Room Size',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
 
)
# define interactive buttons
bathrooms_limit = widgets.IntSlider(
    value = int( data['bathrooms'].mean() ),
    min = data['bathrooms'].min(),
    max = data['bathrooms'].max(),
    step = 1,
    description = 'Minium Values',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
 
)
# define interactive buttons
basement_limit = widgets.IntSlider(
    value = int( data['sqft_basement'].mean() ),
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step = 1,
    description = 'Maximum basements',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
 
)
# define interactive buttons
condition_limit = widgets.IntSlider(
    value = int( data['condition'].mean() ),
    min = data['condition'].min(),
    max = data['condition'].max(),
    step = 1,
    description = 'House Condition',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
 
)
# define interactive buttons
yr_built_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ),
    min = data['yr_built'].min(),
    max = data['yr_built'].max(),
    step = 1,
    description = 'Year Built Limit',
    disable = False,
    style={'description_width': 'initial'} # pega o tamanho total do widget
 
)

In [13]:
def update_map(df, price_limit, living_limit, bathrooms_limit, basement_limit, condition_limit, yr_built_limit ):
    # filter data
    houses = df[ (df['price'] < price_limit) & 
                 (df['sqft_living'] > living_limit) &
                 (df['bathrooms'] > bathrooms_limit) &
                 (df['sqft_basement'] < basement_limit) &
                 (df['condition'] == condition_limit ) &
                 (df['yr_built'] > yr_built_limit)][['id', 'lat', 'long', 'price', 'sqft_living']].copy()
    
    # plot map
    fig = px.scatter_mapbox( houses,
                            lat='lat',
                            lon='long',
                            #color='level',
                            size='price',
                            color_continuous_scale=px.colors.cyclical.IceFire,
                            size_max=15,
                            zoom=10)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r':0, 'l':0, 't':0, 'b':0})
    fig.show()
    

In [14]:
widgets.interactive(update_map, df=fixed(data), price_limit=price_limit, living_limit=living_limit,
                    bathrooms_limit=bathrooms_limit, basement_limit=basement_limit, condition_limit=condition_limit,
                   yr_built_limit=yr_built_limit)

interactive(children=(IntSlider(value=540088, description='Maximum Price', max=7700000, min=75000, step=3, sty…

#### 5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.

In [15]:
#data.dtypes
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

# filtering
date_limit = widgets.SelectionSlider(
    options = data['date'].sort_values().unique().tolist(), # fazer sort para as opções do rádio não ficar desordenadas
    value='2014-12-01',
    description='Max avaliable date',
    disable=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    redout=True
)

year_limit = widgets.SelectionSlider(
    options = data['yr_renovated'].sort_values().unique().tolist(), # fazer sort para as opções do rádio não ficar desordenadas
    value=2000,
    description='Max Year',
    disable=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    redout=True
)

waterfront_limit = widgets.Checkbox(
    value=False,
    description='Is Waterfront ?',
    disable=False,
    indent=False,
)

In [16]:
def update_map(data, date_limit, year_limit, waterfront_limit):
    # filtering data
    df = data[(data['date'] <= date_limit) & 
             (data['yr_renovated'] >= year_limit) &
             (data['waterfront'] == waterfront_limit)].copy()
    
    fig = plt.figure(figsize=(24,12))
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])
    
    # Primeiro gráfico
    by_year = df[['price', 'year']].groupby('year').sum().reset_index()
    sns.barplot(x='year', y='price', data=by_year, ax=ax1)
    
    # Segundo gráfico
    by_day = df[['price', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(x='date', y='price', data=by_day, ax=ax2)
    plt.xticks(rotation=90);
    
    # terceiro gráfico
    by_week_of_year = df[['price', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(x='year_week', y='price', data=by_week_of_year, ax=ax3)        
    plt.xticks(rotation=90);

In [17]:
widgets.interactive(update_map,
                   data = fixed(data),
                   date_limit=date_limit,
                   year_limit=year_limit,
                   waterfront_limit=waterfront_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Max avaliable date', index=212, op…